<a href="https://colab.research.google.com/github/TFortsch/sediment/blob/main/Sediment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
ee.Authenticate()
ee.Initialize(project='ee-fortschthomas52')

In [137]:
import ee
import geemap
import statistics
import numpy as np
from osgeo import gdal
!pip install rasterio
!pip install rasterstats
!pip install statistics
import rasterio
import rasterstats
import statistics

from rasterstats import zonal_stats


In [19]:
sat = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
.filterDate('2021-03-01', '2021-03-31')\
  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10);


In [74]:
balule = ee.Geometry.Polygon(
          [[31.717329298139624, -24.055719459004635],
          [31.716846500516944, -24.057384943478993],
          [31.718235884786658, -24.058736909100467],
          [31.72093418705564, -24.05800214694256],
          [31.71991494762998, -24.054984679571938]])

Map.addLayer(balule, {min: 0, max: 3000}, 'Dekker in River', 1);
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [78]:
SouthAfrica = countries.filter(ee.Filter.eq('ADM0_NAME', 'SouthAfrica'), countries)


In [91]:
 mosaic = sat.median().clip(balule);
display = {'bands': ['B4', 'B3', 'B2'], min: 0, max: 3000};
Map.addLayer(mosaic, display, "RGB", 0);

G = mosaic.select('B3');
IR = mosaic.select('B8');
SWIR = mosaic.select('B11');
R = mosaic.select('B4');

#Normalized Difference Water Index (NDWI) = ( G - IR ) / ( G + IR )
ndwiG = (G.subtract(IR)).divide(G.add(IR));  # Gao
Map.addLayer(ndwiG, {min: -1, max: 1}, 'NDWI_G', 1);

#Normalized Difference Water Index (NDWI) = ( IR - SWIR ) / ( IR + SWIR )
ndwiM = (IR.subtract(SWIR)).divide(SWIR.add(IR));  #McFeeters
Map.addLayer(ndwiM, {min: -1, max: 1}, 'NDWI_M', 1);

#Normalized Difference Water Index (NDWI) = ( G - SWIR ) / ( G + SWIR )
mndwi = (G.subtract(SWIR)).divide(G.add(SWIR));  # Modified
Map.addLayer(mndwi, {min: -1, max: 1}, 'mNDWI', 0);

#Dekker 2002
dekker = (G.add(R)).divide(2);
Map.addLayer(dekker, {min: 0, max: 3000}, 'Dekker', 0);

water = ndwiG.gte(-0.02);
#var wetDekker = dekker.multiply(water);
wetDekker = dekker.updateMask(water) ;
Map.addLayer(wetDekker, {min: 0, max: 3000}, 'Dekker in River', 1);

projection = mosaic.select('B3'). projection (). getInfo() ;

In [95]:
reducers = ee.Reducer.mean().combine({
  'reducer2': ee.Reducer.stdDev(),
  'sharedInputs': True
})

TypeError: Reducer can only be used as a cast to Reducer. Found <class 'dict'>.

In [101]:
stats = wetDekker.reduceRegion({
  reducer= ee.Reducer.mean(),
  geometry: balule,
  scale: 10
})
  best_effort: True,
;

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<ipython-input-101-f70530afc962>, line 2)

In [135]:
import rasterio

#raster = rasterio.open(wetDekker)
mean = zonal_stats(balule, wetDekker, stats="mean")["mean"]

TypeError: invalid path or file: <ee.image.Image object at 0x7cd39913d840>

In [147]:
wdn = to_numeric(wetDekker)


NameError: name 'to_numeric' is not defined

In [150]:
# Install the rasterio module
!pip install rasterio

# Import the necessary modules
import rasterio
from rasterstats import zonal_stats

# Open the wetDekker image as a raster dataset
with rasterio.open(wetDekker.getInfo()['id']) as src:
    # Calculate the mean value of the wetDekker image within the balule geometry
    mean = zonal_stats(balule, src, stats="mean")["mean"]

# Convert the wetDekker image to a NumPy array
wdn = wetDekker.select('B3').array().getInfo()

# Print the mean value and the NumPy array
print(f"Mean value of wetDekker: {mean}")
print(f"NumPy array of wetDekker: {wdn}")

KeyError: 'id'

In [155]:
# Open the wetDekker image as a raster dataset
with rasterio.open(wetDekker.getInfo()['image']) as src:
    # Calculate the mean value of the wetDekker image within the balule geometry
    mean = zonal_stats(balule, src, stats="mean")["mean"]

# Convert the wetDekker image to a NumPy array
wdn = wetDekker.select('B3').array().getInfo()

# Print the mean value and the NumPy array
print(f"Mean value of wetDekker: {mean}")
print(f"NumPy array of wetDekker: {wdn}")

KeyError: 'image'

In [143]:
statistics.mean('wetDekker')

TypeError: can't convert type 'str' to numerator/denominator